# Clustering Crypto

In [51]:
# Initial installs

# !pip install nodejs
# !conda install -c conda-forge nodejs=12 -y
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager --no-build
# !jupyter labextension install jupyterlab-plotly --no-build
# !jupyter labextension install plotlywidget --no-build
# !jupyter labextension install @pyviz/jupyterlab_pyviz --no-build
# !pip install -U altair
# !python3.6 -m pip install --upgrade pip
# %conda install conda
# !pip install vega_datasets




In [52]:
#Initial imports

import requests
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from pathlib import Path
import altair as alt
import numpy as np
from vega_datasets import data

### Fetching Cryptocurrency Data

In [53]:
# Use the following endpoint to fetch json data
#url = "https://min-api.cryptocompare.com/data/all/coinlist"


In [54]:
# # # Alternatively, use the provided csv file:
file_path = Path("crypto_data.csv")

# # Create a DataFrame
crypto_df = pd.read_csv(file_path, index_col=0)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [55]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df[crypto_df['IsTrading'] == True]
crypto_df.head()


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [56]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = crypto_df[crypto_df['Algorithm'] != 'N/A']
crypto_df


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [57]:
# Remove the 'IsTrading' column
crypto_df = crypto_df.drop(columns='IsTrading')
crypto_df.head()


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [58]:
# Remove rows with at least 1 null value
crypto_df = crypto_df.dropna()
crypto_df.head()


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [59]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df[crypto_df['TotalCoinsMined']> 0]
crypto_df.head()


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [60]:
# Drop rows where there are 'N/A' text values
crypto_df = crypto_df[crypto_df!='N/A']
crypto_df.head()


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [61]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df 
coinname = pd.DataFrame(
    data = crypto_df, columns = ['CoinName'])
coinname.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [62]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df.drop(columns='CoinName', inplace=True)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [63]:
# Create dummy variables for text features
X = pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'])
X.head()


,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
# Standardize data
X_scaled = StandardScaler().fit_transform(X)
X_scaled

array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Reducing Dimensions Using PCA

In [65]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
cryptocurrency_pca = pca.fit_transform(X_scaled)


In [66]:
# Create a DataFrame with the principal components data
pcs_df = pd.DataFrame(
    data=cryptocurrency_pca, columns=['principal component 1', 'principal component 2', 'principal component 3'],
    index= crypto_df.index
)
pcs_df.head()


,principal component 1,principal component 2,principal component 3
42,-0.335887,1.039212,-0.401986
404,-0.319247,1.039324,-0.402222
1337,2.323271,1.664603,-0.442668
BTC,-0.144390,-1.346255,0.150821
ETH,-0.154881,-2.054208,0.280304


### Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [67]:
km_scores= []
k = list(range(1, 11))

# Calculate the inertia for the range of k values

for i in k:
    km = KMeans(n_clusters=i, random_state=0).fit(X_scaled)
    preds = km.predict(X_scaled)
    
    print(f'Score for number of cluster(s) {i}: {km.score(X_scaled):.3f}')
    km_scores.append(-km.score(X_scaled))
    
    print('-'*100)



Score for number of cluster(s) 1: -52136.000
----------------------------------------------------------------------------------------------------
Score for number of cluster(s) 2: -51066.744
----------------------------------------------------------------------------------------------------
Score for number of cluster(s) 3: -50002.391
----------------------------------------------------------------------------------------------------
Score for number of cluster(s) 4: -48933.132
----------------------------------------------------------------------------------------------------
Score for number of cluster(s) 5: -48197.176
----------------------------------------------------------------------------------------------------
Score for number of cluster(s) 6: -47362.667
----------------------------------------------------------------------------------------------------
Score for number of cluster(s) 7: -46195.446
-------------------------------------------------------------------------------

In [68]:
# Use the altair scatter plot to create the Elbow Curve.

df_plot = pd.DataFrame({'Number of clusters':[i for i in range(1,11)],'kmean score': km_scores})
alt.Chart(df_plot).mark_line(point=True).encode(
    x=alt.X('Number of clusters:N',axis=alt.Axis(labelAngle=0)),
    y=alt.Y('kmean score',scale=alt.Scale(domain =(43000,53000))),
    tooltip=[alt.Tooltip('kmean score', format='.2f')]
).properties(
    title='Kmean score ifo number of cluster'
).properties(
    title='The elbow method for determining number of clusters',
    width=600
)

alt.Chart(...)

Running K-Means with `k=<your best value for k here>`

In [69]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df=pd.DataFrame({
    'Algorithm': crypto_df.Algorithm,
    'ProofType': crypto_df.ProofType,
    'TotalCoinsMined': crypto_df.TotalCoinsMined,
    'TotalCoinSupply': crypto_df.TotalCoinSupply,
    'PC1': pcs_df['principal component 1'],
    'PC2': pcs_df['principal component 2'],
    'PC3': pcs_df['principal component 3'],
    'coinname': coinname.CoinName,
    'Class': model.labels_,
    },index= crypto_df.index)
clustered_df.head()


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,coinname,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.335887,1.039212,-0.401986,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.319247,1.039324,-0.402222,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.323271,1.664603,-0.442668,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.144390,-1.346255,0.150821,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.154881,-2.054208,0.280304,Ethereum,1


### Visualizing Results

#### Use the altair scatter plot to visualize to visualize the clusters.

In [70]:
# Create a 2D-Scatter with the PCA data and the clusters
#source = data.clustered_df()

alt.Chart(clustered_df).mark_circle(size=60).encode(
    x='PC1', 
    y='PC2', 
    color=alt.Color('Class',scale=alt.Scale(domain=[0,1,2,3],range=['red','blue','orange','yellow'])), 
    tooltip=['coinname', 'Algorithm', 'TotalCoinsMined', 'TotalCoinSupply']).interactive()

alt.Chart(...)

#### Table of Tradable Cryptocurrencies

In [71]:
# Table with tradable cryptos
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    
    display(clustered_df)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,coinname,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.335887,1.039212,-0.401986,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.319247,1.039324,-0.402222,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.323271,1.664603,-0.442668,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.144390,-1.346255,0.150821,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.154881,-2.054208,0.280304,Ethereum,1
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.177112,-1.124626,-0.008684,Litecoin,1
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.361803,1.277234,-0.376009,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.160205,-2.277556,0.306727,Monero,1
ETC,Ethash,PoW,1.133597e+08,210000000,-0.153323,-2.054314,0.280302,Ethereum Classic,1
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.138429,-2.099181,0.273054,ZCash,1


In [72]:
# Print the total number of tradable cryptocurrencies
print(f'The total number of tradable cryptocurrencies is {coinname.count()}')


The total number of tradable cryptocurrencies is CoinName    532
dtype: int64


#### Scatter Plot with Tradable Cryptocurrencies

In [73]:

from sklearn.preprocessing import StandardScaler, MinMaxScaler
# Scale data to create the scatter plot
Min_Max_Scaler = MinMaxScaler()

#Fit the scaler for the Data
ScaledData = Min_Max_Scaler.fit_transform(clustered_df[['TotalCoinSupply','TotalCoinsMined']])

Scaled_df = pd.DataFrame(ScaledData, columns=['TotalCoinSupply','TotalCoinsMined'],index=clustered_df.index)

Scaled_df['CoinName']=clustered_df['coinname']
Scaled_df['Class']=clustered_df['Class']

Scaled_df.head()




,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,1
ETH,0.000000e+00,0.000109,Ethereum,1


In [74]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"

alt.Chart(Scaled_df).mark_circle(size=60).encode(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    color=alt.Color('Class',scale=alt.Scale(domain=[0,1,2,3],range=['red','green','blue','orange'])),
    tooltip=['CoinName','TotalCoinsMined', 'TotalCoinSupply']).interactive()



alt.Chart(...)